In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn
# import torch.nn.utils.prune as prune
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch import Tensor
from torch.nn.parameter import Parameter

import math
import warnings

In [3]:
def quantizeWeight(W,BW):
    W = torch.min(W,(1.0-(2**(-(BW-1.0))))*torch.ones_like(W))
    Wbs = []
    Wbi = torch.lt(W,torch.zeros_like(W)).float()
    Wbs.append(Wbi)
    W = (W + Wbi)
    for i in range(BW-1):
        Wbi = torch.ge(W,0.5*torch.ones_like(W)).float()
        Wbs.append(Wbi)
        W = 2.0*W - Wbi
    carry = torch.ge(W,0.5*torch.ones_like(W)).float()
    for i in range(BW):#-1):
        j = BW-1-i
        Wbs[j] = Wbs[j]+carry
        carry = torch.gt(Wbs[j],1.5*torch.ones_like(Wbs[j])).float()
        Wbs[j] = Wbs[j]*torch.ne(Wbs[j],2.0*torch.ones_like(Wbs[j]))
    return Wbs

def reconstructWeight(Wbs,BW):
    W = torch.zeros_like(Wbs[0])
    for j in range(BW):
        multiplier = (0.5)**j
        if (j == 0):
            multiplier = -1.0
        W += Wbs[j] * multiplier
    return W

def quantize_activations(input):
    input = torch.min(torch.round(input*(2**6))*(2**(-6)) ,(1.0-(2**(-6)))*torch.ones_like(input))
    return input

In [35]:
weight = torch.rand((6,5,3,3))
inputs = torch.rand((1,5,6,6))
bias = torch.rand(6)

weight_q = reconstructWeight(quantizeWeight(weight,6),6)
inputs_q = quantize_activations(inputs)
bias_q = reconstructWeight(quantizeWeight(bias,6),6)

# print(weight_q.size())
# print(weight_q[3:6,:,:,:].size())
# print(weight_q)
# print("a")
# print(weight_q[3:6,:,:,:])
# print("b")
# print(weight_q[0:3,:,:,:])

In [41]:
layer = nn.Conv2d(5,6,3,bias=True)
layer_1 = nn.Conv2d(3,6,3,bias=True)
layer_2 = nn.Conv2d(2,6,3,bias=)
layer.weight.data = weight_q
# layer.bias.data = bias_q
layer_1.weight.data = weight_q[:,0:3,:,:]
# layer.bias.data = torch.zeros(6)
layer_2.weight.data = weight_q[:,3:5,:,:]
# layer_2.bias.data = bias_q


output = layer(inputs)
print(output[:,0,:,:])
# print(output.size())
# print('a')

output_p = layer_1(inputs_q[:,0:3,:,:])
output_p += layer_2(inputs_q[:,3:5,:,:])
# print(output_p.size())
print(output_p[:,0,:,:])
print(output[:,0,:,:] - output_p[:,0,:,:])
# print((output[:,0,:,:] - output_p[:,0,:,:])/output_p[:,0,:,:])
# print(torch.var(output),torch.var(output_p))

tensor([[[12.6413, 12.8813, 11.2089, 10.4253],
         [10.2434,  9.8488, 11.1402, 11.7915],
         [10.8358, 11.0083, 10.7636, 11.4042],
         [10.7552, 10.5342, 12.0006, 12.2040]]], grad_fn=<SliceBackward>)
tensor([[[12.6455, 12.8843, 11.2480, 10.4521],
         [10.2632,  9.8594, 11.1484, 11.8159],
         [10.8491, 11.0171, 10.7954, 11.4214],
         [10.7637, 10.5293, 11.9985, 12.1953]]], grad_fn=<SliceBackward>)
tensor([[[-0.0042, -0.0029, -0.0392, -0.0269],
         [-0.0197, -0.0105, -0.0082, -0.0244],
         [-0.0134, -0.0088, -0.0318, -0.0172],
         [-0.0085,  0.0049,  0.0021,  0.0087]]], grad_fn=<SubBackward0>)
